In [99]:
import pandas as pd
import sqlite3
import esco_utils as eu
import pandas.io.sql as psql

con = sqlite3.connect('data/cache.db')

# Summary 

- Labeling related: 
    - `tags`: The tag table contains individual tags inserted by users.
    - `task_data`: Contains all tasks that are not cancelled. 
    - `projects` and `project_labelers`: project name and id, plus labelers in each project
- Occupations and skills:
    - `occupation`: 
    - `occupations`: 
    - `ISCOGroups`
    - `skill`: skills data, for all languages (in `lang`) Skill types: 'skill/competence', 'knowledge', 'skill/competence knowledge'
    - `skillGroups`: skill groups for all languages (in `lang`)
    - `occupationSkillRelations`: relationships between skills and occupation, can be `essential` or `optional` 
    - `skillSkillRelations`: relations between, can be `essential` or `optional` 

# Labeling

### `tags` table 
If anntator `A1` in response to `task1` confirms tags `1,2,3`, they appear as three rows.
important columns:
- `task_id`: the task 
- `occupation_id` & `occupation_title`: the id & title of the occupation tag chosen by user
- `inserted_at` & `label_time`: time for the each tag

In [100]:
pd.read_sql("SELECT * FROM tags",con).head(1)

,index,inserted_at,label_time,labeler_id,lang,task_id,occupation_id,occupation_title
0,0,"Wed, 12 Jan 2022 14:04:25 GMT",8347,61dda30da33565ec1ce31aab,None,61dde40c527776b760a85fa0,1864,sales assistant


### `task_data` table
Important columns: 
- `project_id` and `project_name`: id & name of the project for this task
- `_id`: the task id 
- `title` and `description`: the title and description of the task. 

In [83]:
pd.read_sql("SELECT * FROM task_data",con).head(1)

,project_name,project_id,_id,created_at,status,task-type,total_labels,updated_at,description,title
0,United Kingdom,61ddbe23527776b760a84bdd,61dddc3e527776b760a85c18,"Tue, 11 Jan 2022 19:36:30 GMT",pending,esco-text-tagging,0,"Tue, 11 Jan 2022 19:36:30 GMT",We're pleased to announce that SThree is looki...,Credit Controller - DACH


### `project` and `project_labelers`
get project name and id, plus labelers in each project

In [82]:
pd.read_sql("SELECT * FROM projects",con).head(1)

,created_at,icon_id,labels_per_task,model_id,project_id,project_name,updated_at,lang,total_complete_tasks,total_labels,total_tasks,total_tasks_labeled
0,"Tue, 11 Jan 2022 14:14:51 GMT",61dd99d3527776b760a8497a,3,7693d764-72e5-11ec-83e4-0242c0a8d007,61dd90dbce271774889fbc02,GB,"Tue, 11 Jan 2022 14:14:51 GMT",en,0,1,300,0


In [81]:
pd.read_sql("SELECT * FROM project_labelers",con).head(1)

,project_id,labeler_status,_id,email
0,61dd90dbce271774889fbc02,active_labelers,,ari@gmail.com


# Occupations and skills 

### `occupations`: title, description, and ISCO codes in all languages
contain title, alternative title, and description of all ISCO occupations in all EU languages. 
Important coluns:
- `occupation_id`: same occupation_id used in the `tags` table 
- `iscoGroup`: the group corresponding of this occupation, same as `code` in `ISCOGroups` table
- `preferredLabel`: title
- `altLabels`: alternative titles, seperated by `\n`
- `description`: description
- `lang`: the language for the record
- `conceptUri`: the url to the website for this resource

In [80]:
lang = 'en'
pd.read_sql(f"SELECT * FROM occupations WHERE lang='{lang}'",con).head(1)

,conceptType,conceptUri,iscoGroup,preferredLabel,altLabels,hiddenLabels,status,modifiedDate,regulatedProfessionNote,scopeNote,definition,inScheme,description,lang,esco_version,occupation_id,external_id
0,Occupation,http://data.europa.eu/esco/occupation/00030d09...,2166,technical director,technical and operations director\nhead of tec...,None,released,2016-07-05T13:58:41Z,http://data.europa.eu/esco/regulated-professio...,None,None,http://data.europa.eu/esco/concept-scheme/occu...,Technical directors realise the artistic visio...,en,v1.0.3,2,http://data.europa.eu/esco/occupation/00030d09...


### `ISCOGroups`: the groups of occupations 
Columns: 
- `code`: code of thiso group
- `preferredLabel`: the title 
- `altLabels`: the alternative title

__Hierarchy__: the hierarchy of ISCO groups is preserved in their `code`. Example: ISCO group with code `22` is "Health professionals", which is the parent of group "Medical doctors" with code `221`, which is a parent of "Specialist medical practitioners" with code `2212". 

In [115]:
pd.read_sql(f"SELECT * FROM ISCOGroups WHERE lang='{lang}' AND code='2212'",con)

,conceptType,conceptUri,code,preferredLabel,altLabels,inScheme,description,lang,esco_version
0,ISCOGroup,http://data.europa.eu/esco/isco/C2212,2212,Specialist medical practitioners,None,http://data.europa.eu/esco/concept-scheme/occu...,Specialist medical practitioners (medical doct...,en,v1.0.3


### Skill data

In [90]:
skill = pd.read_sql(f"SELECT * FROM skill WHERE lang='{lang}'",con)
skillG = pd.read_sql(f"SELECT * FROM skillGroups WHERE lang='{lang}'",con)
occ2skill = pd.read_sql("SELECT * FROM occupationSkillRelations", con)
skill2skill = pd.read_sql("SELECT * FROM skillSkillRelations", con)
adjSkill = pd.read_sql("SELECT * FROM transversalSkillsCollection", con)